In [52]:
import json
with open('../Benchmarks/Full.json', 'r') as f:
    data = json.load(f)

In [53]:
eqs = []
for g in data:
    for e in g['examples']:
        eqs.append(e['eq'])

lengths = [len(e) for e in eqs]

In [54]:
min(lengths)

5

In [55]:
max(lengths)

166

In [56]:
from statistics import mean
mean(lengths)

18.10561797752809

Comparison experiment

In [57]:
from common import parse_log

result_data = {}
time_data = {}
for config in range(1, 5): # C1..C4
    with open(f'run_EqFix_C{config}.log', 'r') as f:
        result_data[(config, 'E')], time_data[(config, 'E')] = parse_log(f.readlines())

    with open(f'run_FlashFill_C{config}.log', 'r') as f:
        result_data[(config, 'F')], time_data[(config, 'F')] = parse_log(f.readlines())

def max_of(xs):
    return None if len(xs) == 0 else max(xs)

EqFix ranking:
1. At most, how many rules were attempted on solved test cases?

In [58]:
[max_of([v for v in result_data[(config, 'E')].values() if v is not None]) for config in range(1, 5)]

[5, 5, 5, 5]

2. How many were solved by the top-ranked rule?

In [59]:
[len([1 for v in result_data[(config, 'E')].values() if v == 1]) for config in range(1, 5)]

[57, 62, 66, 68]

FlashFill ranking:
1. At most, how many rules were attempted on solved test cases?

In [60]:
[max_of([v for v in result_data[(config, 'F')].values() if v is not None]) for config in range(1, 5)]

[None, 8, 8, 8]

2. How many were solved by the top-ranked rule?

In [61]:
[len([1 for v in result_data[(config, 'F')].values() if v == 1]) for config in range(1, 5)]

[0, 8, 21, 33]

EqFix synthesis time:

In [62]:
E_times = list(time_data[(4, 'E')].values())
mean(E_times)

200.83229438202247

FlashFill synthesis time:

In [63]:
F_times = list(time_data[(4, 'F')].values())
mean(F_times)

476.3707865168539

EqFix spent less time on how many?

In [64]:
assert len(E_times) == len(F_times) == 89
len([1 for i in range(len(E_times)) if E_times[i] < F_times[i]])

78

Relaxer

In [65]:
import xml.etree.ElementTree as ET
tree = ET.parse('../EqFix/bin/Release/netcoreapp2.0/1.xml')
root = tree.getroot()
relaxers = root.findall('value/items/array/elem/Item2')
assert len(relaxers) == 82

count = 0
for r in relaxers:
    text = r.findtext('elem/Case')
    if text and 'Relax' in text:
        count += 1

count

17